In [1]:
import pandas as pd

In [44]:
df = pd.read_parquet('green_tripdata_2025-11.parquet')

In [3]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [4]:
df.dtypes # no need for pd.to_datetime

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [5]:
mask = (
    (df['lpep_pickup_datetime'] >= '2025-11-01') &
    (df['lpep_pickup_datetime'] < '2025-12-01') &
    (df['trip_distance'] <= 1)
)

result_count = df[mask].shape[0]

print(f"Number of trips: {result_count}")

Number of trips: 8007


In [14]:
max_distance = df[df['trip_distance'] <= 100]['trip_distance'].max()
df[df['trip_distance'] == max_distance]['lpep_pickup_datetime']

18867   2025-11-14 15:36:27
Name: lpep_pickup_datetime, dtype: datetime64[us]

In [16]:
zones = pd.read_csv('taxi_zone_lookup.csv')

In [41]:
zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [45]:
# Merge for pick-up location
df = df.merge(zones[['LocationID', 'Zone']], left_on='PULocationID', right_on='LocationID', how='left')
df = df.rename(columns={'Zone': 'PU_zone'})
df = df.drop('LocationID', axis=1)

# Merge for drop-off location
df = df.merge(zones[['LocationID', 'Zone']], left_on='DOLocationID', right_on='LocationID', how='left')
df = df.rename(columns={'Zone': 'DO_zone'})
df = df.drop('LocationID', axis=1)

In [47]:
df_11_18 = df[df['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()]
popular_zones = df_11_18.groupby('PU_zone').size().reset_index(name='trip_count').sort_values(by='trip_count', ascending=False)
print(popular_zones)

                          PU_zone  trip_count
39              East Harlem North         434
40              East Harlem South         298
23                   Central Park         104
85            Morningside Heights          88
52                   Forest Hills          72
..                            ...         ...
24                      Chinatown           1
97   Penn Station/Madison Sq West           1
100               Queensboro Hill           1
18                Cambria Heights           1
68            Kingsbridge Heights           1

[136 rows x 2 columns]


In [55]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,PU_zone,DO_zone
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00,East Harlem North,Central Harlem North
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00,East Harlem North,Central Harlem North
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00,Elmhurst/Maspeth,Middle Village
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00,Morningside Heights,Inwood
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00,Morningside Heights,Yorkville East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,None,NaN,59,51,NaN,8.50,33.22,...,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00,Crotona Park,Co-Op City
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,None,NaN,74,151,NaN,1.73,13.86,...,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00,East Harlem North,Manhattan Valley
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,None,NaN,33,163,NaN,7.52,38.42,...,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75,Brooklyn Heights,Midtown North
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,None,NaN,16,95,NaN,5.61,24.67,...,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00,Bayside,Forest Hills


In [56]:
east_harlem_north_df = df[df['PU_zone'] == 'East Harlem North']
largest_tip = east_harlem_north_df.loc[east_harlem_north_df['tip_amount'].idxmax()]
print(largest_tip['DO_zone'])

Yorkville West
